In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

Loading Dataset through google drive


In [82]:
Dataset1 = '/content/drive/MyDrive/DataSet_ML/Movie Recomendation/tmdb_5000_credits.csv'
Dataset2 = '/content/drive/MyDrive/DataSet_ML/Movie Recomendation/tmdb_5000_movies.csv'

In [83]:
df_1 = pd.read_csv(Dataset1)
df_2 = pd.read_csv(Dataset2)

Features of dataset 1

In [84]:
df_1.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

Features of dataset 2

In [85]:
df_2.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [86]:
df_1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [87]:
df_2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


Created a new dataframe called 'new_df', which has important features related to our analysis.
1. Movie_id = Important to track moves
2. title = important as name of movie
3. cast = Cast is very important for some people to select movies
4. Crew = Here director name is very prominent feature
5. genres = importat as which type of movies it is
6. keywords = used for suggestion and finding of movie
7. overview = to know the summery of movies
8. Production_companies = imporant for some viewers

In [88]:
new_df = pd.concat([df_1[['movie_id', 'title', 'cast', 'crew']], df_2[['genres', 'keywords', 'overview', 'production_companies']]], axis = 1)

In [89]:
new_df.shape

(4803, 8)

Checking if there any null value.

In [90]:
new_df.isnull().sum()

movie_id                0
title                   0
cast                    0
crew                    0
genres                  0
keywords                0
overview                3
production_companies    0
dtype: int64

Here only three null values is found in overview so dropping that will not affect much as we have 4803 X 3 dataset

In [91]:
#dropping the null values
new_df.dropna(inplace=True)

In [92]:
# Checked is there any duplicate values
new_df.duplicated().sum()

0

In [93]:
new_df

,movie_id,title,cast,crew,genres,keywords,overview,production_companies
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,"[{""name"": ""Columbia Pictures"", ""id"": 5}]"
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,[]
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""name"": ""Front Street Pictures"", ""id"": 3958}..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[],[],When ambitious New York attorney Sam is sent t...,[]


In [94]:
new_df.iloc[1].genres

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

We have featues in string list so converting it into in list datatype

In [95]:
# function to convert string into list
def list_convert(string_list):
  my_list = ast.literal_eval(string_list)
  new_list = []
  for kewords in my_list:
    new_list.append(kewords['name'])
  return new_list

Converting all features into list using list_convert function

In [96]:
new_df['genres'] = new_df['genres'].apply(list_convert)

In [97]:
new_df.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [98]:
new_df['cast'] = new_df['cast'].apply(list_convert)

In [99]:
new_df.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [100]:
new_df['keywords'] = new_df['keywords'].apply(list_convert)

In [101]:
new_df.iloc[0].production_companies

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [102]:
new_df['production_companies'] = new_df['production_companies'].apply(list_convert)

In [103]:
new_df.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [104]:
# function to find the director form crew feature
def list_convert_director(string_list):
  my_list = ast.literal_eval(string_list)
  new_list = []
  for kewords in my_list:
    if kewords['job'] == 'Director':
      new_list.append(kewords['name'])
      break
  return new_list

In [105]:
new_df['crew'] = new_df['crew'].apply(list_convert_director)

In [106]:
new_df.iloc[0].overview

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [107]:
def list_make(string):
  new_list = [string]
  return new_list

In [108]:
new_df['overview'] = new_df['overview'].apply(list_make)

Removing the space between the names and words

In [109]:
new_df['cast'] = new_df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
new_df['crew'] = new_df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
new_df['genres'] = new_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
new_df['keywords'] = new_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
# new_df['overview'] = new_df['overview'].apply(lambda x: [i.replace(" ", "") for i in x])
new_df['production_companies'] = new_df['production_companies'].apply(lambda x: [i.replace(" ", "") for i in x])


In [110]:
new_df['tags'] = new_df['cast'] + new_df['crew'] + new_df['genres'] + new_df['keywords'] + new_df['overview'] + new_df['production_companies']

In [111]:
new_df

,movie_id,title,cast,crew,genres,keywords,overview,production_companies,tags
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In the 22nd century, a paraplegic Marine is d...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain Barbossa, long believed to be dead, h...","[WaltDisneyPictures, JerryBruckheimerFilms, Se...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",[A cryptic message from Bond’s past sends him ...,"[ColumbiaPictures, Danjaq, B24]","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp..."
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",[Following the death of District Attorney Harv...,"[LegendaryPictures, WarnerBros., DCEntertainme...","[ChristianBale, MichaelCaine, GaryOldman, Anne..."
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John Carter is a war-weary, former military c...",[WaltDisneyPictures],"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi..."
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez],"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...",[El Mariachi just wants to play his guitar and...,[ColumbiaPictures],"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,..."
4799,72766,Newlyweds,"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns],"[Comedy, Romance]",[],[A newlywed couple's honeymoon is upended by t...,[],"[EdwardBurns, KerryBishé, MarshaDietlein, Cait..."
4800,231617,"Signed, Sealed, Delivered","[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith],"[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[""Signed, Sealed, Delivered"" introduces a dedi...","[FrontStreetPictures, MuseEntertainmentEnterpr...","[EricMabius, KristinBooth, CrystalLowe, GeoffG..."
4801,126186,Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc...",[DanielHsia],[],[],[When ambitious New York attorney Sam is sent ...,[],"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc..."


Final_df is made by using three features movie_id, title and tags, which is made by concatination of list of others all featuers

In [112]:
final_df = new_df[['movie_id', 'title', 'tags']]
final_df

,movie_id,title,tags
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp..."
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne..."
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi..."
...,...,...,...
4798,9367,El Mariachi,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,..."
4799,72766,Newlyweds,"[EdwardBurns, KerryBishé, MarshaDietlein, Cait..."
4800,231617,"Signed, Sealed, Delivered","[EricMabius, KristinBooth, CrystalLowe, GeoffG..."
4801,126186,Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc..."


In [113]:
final_df['tags'] = final_df['tags'].apply(lambda x: " ".join(x))
final_df

<ipython-input-113-0d8f55d9c1dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tags'] = final_df['tags'].apply(lambda x: " ".join(x))


,movie_id,title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver Step...
1,285,Pirates of the Caribbean: At World's End,JohnnyDepp OrlandoBloom KeiraKnightley Stellan...
2,206647,Spectre,DanielCraig ChristophWaltz LéaSeydoux RalphFie...
3,49026,The Dark Knight Rises,ChristianBale MichaelCaine GaryOldman AnneHath...
4,49529,John Carter,TaylorKitsch LynnCollins SamanthaMorton Willem...
...,...,...,...
4798,9367,El Mariachi,CarlosGallardo JaimedeHoyos PeterMarquardt Rei...
4799,72766,Newlyweds,EdwardBurns KerryBishé MarshaDietlein CaitlinF...
4800,231617,"Signed, Sealed, Delivered",EricMabius KristinBooth CrystalLowe GeoffGusta...
4801,126186,Shanghai Calling,DanielHenney ElizaCoupe BillPaxton AlanRuck Zh...


**Stemming**

Before stemming

In [114]:
final_df['tags'][0]

'SamWorthington ZoeSaldana SigourneyWeaver StephenLang MichelleRodriguez GiovanniRibisi JoelDavidMoore CCHPounder WesStudi LazAlonso DileepRao MattGerald SeanAnthonyMoran JasonWhyte ScottLawrence KellyKilgour JamesPatrickPitt SeanPatrickMurphy PeterDillon KevinDorman KelsonHenderson DavidVanHorn JacobTomuri MichaelBlain-Rozgay JonCurry LukeHawker WoodySchultz PeterMensah SoniaYee JahnelCurfman IlramChoi KylaWarren LisaRoumain DebraWilson ChrisMala TaylorKibby JodieLandau JulieLamm CullenB.Madden JosephBradyMadden FrankieTorres AustinWilson SaraWilson TamicaWashington-Miller LucyBriant NathanMeister GerryBlair MatthewChamberlain PaulYates WrayWilson JamesGaylyn MelvinLenoClarkIII CarvonFutrell BrandonJelkes MicahMoch HanniyahMuhammad ChristopherNolen ChristaOliver AprilMarieThomas BravitaA.Threatt ColinBleasdale MikeBodnar MattClayton NicoleDionne JamieHarrison AllanHenry AnthonyIngruber AshleyJeffery DeanKnowsley JosephMika-Hunt TerryNotary KaiPantano LoganPithyou StuartPollock Raja Ga

In [115]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [116]:
ps = PorterStemmer()

In [117]:
def stemmer_sentenses(string):
  words_in_sentence = word_tokenize(string)
  # print(words_in_sentence)
  steemer_words = []
  for w in words_in_sentence:
      steemer_words.append(ps.stem(w))
  return " ".join(steemer_words)

In [118]:
final_df['tags'] = final_df['tags'].apply(stemmer_sentenses)

<ipython-input-118-bcba362bcf53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tags'] = final_df['tags'].apply(stemmer_sentenses)


After Stemming

In [119]:
final_df['tags'][0]

'samworthington zoesaldana sigourneyweav stephenlang michellerodriguez giovanniribisi joeldavidmoor cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyt scottlawr kellykilgour jamespatrickpitt seanpatrickmurphi peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurri lukehawk woodyschultz petermensah soniaye jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibbi jodielandau julielamm cullenb.madden josephbradymadden frankietorr austinwilson sarawilson tamicawashington-mil lucybri nathanmeist gerryblair matthewchamberlain paulyat wraywilson jamesgaylyn melvinlenoclarkiii carvonfutrel brandonjelk micahmoch hanniyahmuhammad christophernolen christaoliv aprilmariethoma bravitaa.threatt colinbleasdal mikebodnar mattclayton nicoledionn jamieharrison allanhenri anthonyingrub ashleyjefferi deanknowsley josephmika-hunt terrynotari kaipantano loganpithy stuartpollock raja garethruck rhiansheehan t.j.storm jod

**Performing text vectorization**

In [120]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [121]:
final_df_vectores = cv.fit_transform(final_df['tags']).toarray()

In [122]:
final_df_vectores

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [123]:
for i in cv.get_feature_names_out():
  print(i)

000
007
10
100
11
12
13
14
1492pictur
15
16
17
18
18th
19
1920
1930
1940
1950
1960
1970
1980
1990
19th
19thcenturi
20
20th
21lapsentertain
24
25
2929product
30
3artsentertain
3d
40
40acr
50
60
70
aaron
aaroneckhart
aarontaylor
aasifmandvi
abandon
abduct
abigailbreslin
abil
abl
abov
abus
academi
accept
access
accid
accident
acclaim
accompani
accomplish
account
accus
ace
achiev
act
action
activ
activist
actor
actress
actual
adam
adambrodi
adamgoldberg
adamlefevr
adamsandl
adamscott
adamshankman
adapt
add
addict
adewaleakinnuoye
adjust
admir
admit
adolesc
adopt
ador
adrianmartinez
adrienbrodi
adult
adulteri
adulthood
advanc
advantag
adventur
advertis
advic
advis
affair
affect
afghanistan
africa
african
aftercreditssting
afterlif
aftermath
ag
agbaj
age
agediffer
agenc
agenda
agent
aggress
ago
agre
ahead
aid
aidanquinn
ail
aim
air
airplan
airport
al
alanalda
alanarkin
alancum
alanrickman
alantudyk
alaska
albert
albertbrook
albertfinney
alcohol
alconentertain
alecbaldwin
alessandronivola
ale

In [124]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

Calculating Cosine similarity between all vectors

In [125]:
cosine_similirity = cosine_similarity(final_df_vectores)

In [126]:
cosine_similirity

array([[1.        , 0.06451986, 0.07194017, ..., 0.05847609, 0.02150662,
        0.        ],
       [0.06451986, 1.        , 0.07271792, ..., 0.01970276, 0.        ,
        0.02248469],
       [0.07194017, 0.07271792, 1.        , ..., 0.04393748, 0.        ,
        0.        ],
       ...,
       [0.05847609, 0.01970276, 0.04393748, ..., 1.        , 0.05910828,
        0.04075696],
       [0.02150662, 0.        , 0.        , ..., 0.05910828, 1.        ,
        0.08993875],
       [0.        , 0.02248469, 0.        , ..., 0.04075696, 0.08993875,
        1.        ]])

In [127]:
sorted(list(enumerate(cosine_similirity[4])), reverse=True, key = lambda x:x[1])

[(4, 1.0000000000000002),
 (3088, 0.2678998598884099),
 (1319, 0.2579015967415503),
 (3372, 0.25636458762546666),
 (610, 0.24553427732134045),
 (27, 0.23783535600422523),
 (270, 0.2341477628019854),
 (193, 0.231416764751961),
 (939, 0.23130179953048294),
 (4377, 0.22868813597171714),
 (752, 0.22821069928152787),
 (313, 0.2279211529192759),
 (1937, 0.2253523886262605),
 (207, 0.223606797749979),
 (2427, 0.2233587590738024),
 (4442, 0.2224936802220051),
 (1254, 0.22247460415730488),
 (4141, 0.21869176243407468),
 (3808, 0.21793430611327824),
 (2653, 0.21591675854376524),
 (2955, 0.2149948634112309),
 (3223, 0.2149948634112309),
 (1044, 0.21452908258025827),
 (3128, 0.21155435413917806),
 (1140, 0.21130685627241338),
 (1523, 0.21105794120443455),
 (2977, 0.21058465757133069),
 (2550, 0.20976176963403032),
 (60, 0.20975106832189755),
 (1959, 0.2095139706465989),
 (1008, 0.20951397064659888),
 (697, 0.20720853716760057),
 (3660, 0.20706998246326275),
 (20, 0.20586558950390912),
 (1290, 0.20

Fetching the top 10 movies on basis of cosine similarity

In [128]:
def movie_recomened(movie_title):
  if movie_title in final_df['title'].values:
    suggested_movies = []
    movie_index = final_df[final_df['title'] == movie_title].index[0]
    distance = cosine_similirity[movie_index]
    movies_list = sorted(list(enumerate(distance)), reverse=True, key = lambda x:x[1])[1:11]
    for movie_names in movies_list:
      suggested_movies.append(new_df.iloc[movie_names[0]].title)
    return suggested_movies
  else:
    print("No such movie found")


Function to print all 10 suggested movies

In [160]:
def find_movies():
  movies = input("Enter the name of Movie: ")
  print("Top 10 Suggested movies are: \n")
  if movie_recomened(movies) is not None:
    for i, j in enumerate(movie_recomened(movies)):
      print(i+1, j)

Run the cell to find movies

In [157]:
find_movies()

Enter the name of Movie: Underworld: Awakening
Top 10 Suggested movies are: 

1 Underworld: Evolution
2 Underworld
3 Underworld: Rise of the Lycans
4 Blade II
5 Daybreakers
6 BloodRayne
7 The Dead Undead
8 Abraham Lincoln: Vampire Hunter
9 Archaeology of a Woman
10 The Age of Adaline


Run the cell to find movies

In [162]:
find_movies()

Enter the name of Movie: Beverly Hills Cop III
Top 10 Suggested movies are: 

1 Beverly Hills Cop
2 Beverly Hills Cop II
3 Tango & Cash
4 Big Mommas: Like Father, Like Son
5 National Lampoon's Loaded Weapon 1
6 Stone Cold
7 Bad Boys
8 Bad Boys II
9 Narc
10 The Italian Job


**List of all Movies**

In [154]:
final_df['title'].tolist()

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',